In [1]:
import requests                # Include HTTP Requests module
from bs4 import BeautifulSoup  # Include BS web scraping module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select

In [14]:
url = "http://d3hoops.com/seasons/men/2019-20/boxscores/20200106_pwnb.xml?view=plays"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
soup.prettify()
#home = soup.find_all('tr', attrs = {'class':'row home'})
#away = soup.find_all('tr', attrs = {'class':'row visitor'})
#gameid = url.replace('//','/').split('/')[6].split('?')[0].replace('.xml','')

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <title>\n   ERROR: The request could not be satisfied\n  </title>\n </head>\n <body>\n  <h1>\n   403 ERROR\n  </h1>\n  <h2>\n   The request could not be satisfied.\n  </h2>\n  <hr noshade="" size="1px"/>\n  Request blocked.\nWe can\'t connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.\n  <br clear="all"/>\n  If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.\n  <br clear="all"/>\n  <hr noshade="" size="1px"/>\n  <pre>\nGenerated by cloudfront (CloudFront)\nRequest ID: bacyph3tMkQ9Q2W8FWmE2ny9JaZj9I_omcPcA_-v9pt9rzuNAU1UKw==\n</pre>\n  <address>\n  </address>\n </bod

In [7]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
for i in games.c:
    print(i.name,i.type)
stmt = select([games]).where(games.columns.id == gameid)
#results = conn.execute(stmt).fetchall()
if not results:
    conn.execute(games.insert(), id=gameid)
stmt = select([games]).where(games.columns.id == gameid)
#results = conn.execute(stmt).fetchall()
#results

id VARCHAR
home INTEGER
homeScore INTEGER
away INTEGER
awayScore INTEGER


[('20200106_pwnb', None, None, None, None)]

In [8]:
dfH = pd.DataFrame()
for i in home[0:30]:
  t = i.find('td',attrs={'class':'time'}).get_text().strip()
  x = i.find('span',attrs={'class':'text'}).get_text().strip()
  sc = i.find('td', attrs={'class':'score'}).get_text().strip()
  scA = 0
  scH = 0
  if sc:
    scH = sc.split('-')[0]
    scA = sc.split('-')[1]
  dfH = pd.concat([dfH,pd.DataFrame([[t,x,scH,scA]],columns=['gameTime','action','scoreHome','scoreAway'])])
dfH['team'] = 'Home'

dfA = pd.DataFrame()
for i in away[0:30]:
  t = i.find('td',attrs={'class':'time'}).get_text().strip()
  x = i.find('span',attrs={'class':'text'}).get_text().strip()
  sc = i.find('td', attrs={'class':'score'}).get_text().strip()
  scA = 0
  scH = 0
  if sc:
    scH = sc.split('-')[0]
    scA = sc.split('-')[1]
  dfA = pd.concat([dfA,pd.DataFrame([[t,x,scH,scA]],columns=['gameTime','action','scoreHome','scoreAway'])])
dfA['team'] = 'Away'

df = pd.concat([dfH,dfA])

df['player'] = df.action.str.extract('([^a-z]{2,})')
df['action'] = df.action.str.replace('([^a-z0-9\._]{2,})','')#.str.split()

df = df.set_index('gameTime')
df = df.sort_index(ascending=False)

df['time1'] = df.index

df['Time_diff'] = pd.to_datetime(df['time1'].astype(str)).diff(-1).dt.total_seconds().div(60)

#df = df.reset_index()

#df.head(10)
#df['action'].unique() #create lookup table

NameError: name 'home' is not defined

In [39]:
conn.close()